# Econ 8210 Quant Macro, Homework 1
## Part 2 - Solution Methods
Haosi Shen, Fall 2024

In [5]:
# Housekeeping
import numpy as np
import pandas as pd
import time
import torch

np.random.seed(42) 

## Computing Pareto Efficient Allocations

Consider an endowment economy with $m$ different goods and $n$ agents. Each agent $i = 1, ..., n$ has an endowment $e_j^i >0$ for every $j = 1, ..., m$ and a utility function of the form
$$ u^i (x) = \sum_{j=1}^{m} \alpha_j \frac{x_{j}^{1+\omega_j^i}}{1+\omega_j^i} $$
where $\alpha_j > 0 > \omega_j^i$ are agent-specific parameters.

Given some social weights $\lambda_i > 0$, solve for the social planner’s problem for $m = n = 3$ using the **Adam (Adaptive Moment Estimation)** method. Try different values of $\alpha_j,\;\omega_j^i,\;\lambda_i$. 

$$ \max_{\{x^i\}} \; \sum_{i=1}^{n} \lambda_i u^{i}(x)$$

Compute first the case where all the agents have the same parameters and
social weights and later a case where there is a fair degree of heterogeneity.

How does the method perform? How does heterogeneity in the agent-specific parameters
affect the results?

Can you handle the case where $m = n = 10$?

> I choose to use **Adam** since it is more efficient for high-dimensional optimization problems and offers more stability and robustness. However, if we are only solving for the case of $m=n=3$, then the Newton-Raphson method might be more ideal since this problem is relatively smooth. Adam only requires gradient information and does not involve inverting the Hessian.

In [9]:
# utility function for agent i
def utility(x, alpha, omega):
    return torch.sum(alpha * (x ** (1 + omega)) / (1 + omega))

# Social planner's objective
def social_planner_objective(x, alpha_j, omega_j, lambda_i):
    total_utility = 0
    for i in range(n):
        total_utility += lambda_i[i] * utility(x[i], alpha_j, omega_j)
    return -total_utility   # maximization, so take negative

### Case I: Homogeneous Agents

> $m = n = 3$

All agents $j$ have the same parameters $\alpha_j, \omega_j^i$ and social weights $\lambda_j$. 

In [10]:
import torch.optim as optim

# =============== DEFINE PARAMETERS ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32) 
omega_j = torch.tensor([[0.5, 0.5, 0.5]] * n, dtype = torch.float32)
lambda_i = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32)  # Pareto weights

# initial allocations
x_i = torch.randn((n, m), requires_grad = True)
# Set up optimizer
optimizer = optim.Adam([x_i], lr=0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective(x_i, alpha_j, omega_j, lambda_i)
    objective.backward()
    optimizer.step()

    if iteration % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
print("Optimal allocations:")
print(x_i.detach().numpy())


Iteration 0: Objective = nan
Iteration 100: Objective = nan
Iteration 200: Objective = nan
Iteration 300: Objective = nan
Iteration 400: Objective = nan
Iteration 500: Objective = nan
Iteration 600: Objective = nan
Iteration 700: Objective = nan
Iteration 800: Objective = nan
Iteration 900: Objective = nan
Optimal allocations:
[[13.359219       nan       nan]
 [13.528759 13.40121        nan]
 [13.581224 13.375893 13.401505]]


### Case II: Heterogeneous Agents 
> $m = n = 3$

Parameters $\alpha_j, \omega_j^i$
Social weights $\lambda_j$

In [ ]:
# =============== DEFINE PARAMETERS ===============
alpha_j_het = torch.tensor([1.0, 0.8, 1.2], dtype = torch.float32) 
omega_j_het = torch.tensor([[0.3, 0.5, 0.7]] * n, dtype = torch.float32)
lambda_i_het = torch.tensor([0.9, 1.0, 1.01], dtype = torch.float32)  # Pareto weights

